# 🗂️ SESIÓN 1: HDFS OPERACIONES AVANZADAS
## Análisis y Optimización de HDFS

### 🎯 **OBJETIVO**
Aprender operaciones avanzadas de HDFS y técnicas de optimización

### 📋 **CONTENIDO**
- Análisis de bloques y replicación
- Optimización de almacenamiento
- Monitoreo y recuperación

In [ ]:
# 🔧 CONFIGURACIÓN INICIAL
import subprocess
import os
import json
from hdfs import InsecureClient
from datetime import datetime

# Conectar a HDFS
hdfs_client = InsecureClient('http://namenode:9870')

# Crear directorio para análisis
analysis_dir = '/analisis_avanzado'
hdfs_client.makedirs(analysis_dir)

print("🔗 Conexión HDFS establecida")
print(f"📁 Directorio de análisis: {analysis_dir}")

print("✅ Configuración completada")

## 📝 EJERCICIO 1: Análisis de Bloques

Analizaremos la distribución de bloques y replicación en HDFS

In [ ]:
# 1.1 Crear archivo de prueba grande
print("📄 CREANDO ARCHIVO DE PRUEBA")
print("=========================")

try:
    # Crear archivo local grande
    test_file = 'test_large.txt'
    with open(test_file, 'w') as f:
        for i in range(1000000):  # 1 millón de líneas
            f.write(f"Línea de prueba {i}\n")
    
    # Subir a HDFS
    hdfs_path = f'{analysis_dir}/test_large.txt'
    hdfs_client.upload(hdfs_path, test_file)
    print(f"✅ Archivo subido a {hdfs_path}")
    
except Exception as e:
    print(f"❌ Error: {e}")

print("\n✅ Archivo creado y subido")

In [ ]:
# 1.2 Analizar distribución de bloques
print("📊 ANALIZANDO DISTRIBUCIÓN DE BLOQUES")
print("===============================")

try:
    # Obtener información de bloques
    result = subprocess.run(['hdfs', 'fsck', hdfs_path, '-files', '-blocks', '-locations'], 
                          capture_output=True, text=True)
    
    if result.returncode == 0:
        print("Información de bloques:")
        print(result.stdout)
        
        # Extraer estadísticas
        lines = result.stdout.split('\n')
        for line in lines:
            if 'Total blocks' in line:
                print(f"\n📊 {line}")
            elif 'Average block size' in line:
                print(f"📊 {line}")
    else:
        print(f"❌ Error: {result.stderr}")
        
except Exception as e:
    print(f"❌ Error: {e}")

print("\n✅ Análisis completado")

## 📝 EJERCICIO 2: Optimización de Almacenamiento

Implementaremos técnicas de optimización de almacenamiento

In [ ]:
# 2.1 Configurar factor de replicación
print("⚙️ CONFIGURANDO FACTOR DE REPLICACIÓN")
print("================================")

try:
    # Cambiar factor de replicación
    result = subprocess.run(['hdfs', 'dfs', '-setrep', '-w', '2', hdfs_path], 
                          capture_output=True, text=True)
    
    if result.returncode == 0:
        print("✅ Factor de replicación actualizado")
        print(result.stdout)
    else:
        print(f"❌ Error: {result.stderr}")
        
except Exception as e:
    print(f"❌ Error: {e}")

print("\n✅ Configuración completada")

In [ ]:
# 2.2 Implementar compresión
print("🗜️ IMPLEMENTANDO COMPRESIÓN")
print("========================")

try:
    # Crear archivo comprimido
    compressed_path = f'{analysis_dir}/test_compressed.gz'
    
    # Comprimir archivo
    result = subprocess.run(['hdfs', 'dfs', '-copyFromLocal', '-f', test_file, compressed_path], 
                          capture_output=True, text=True)
    
    if result.returncode == 0:
        # Verificar tamaños
        original_size = hdfs_client.status(hdfs_path)['length']
        compressed_size = hdfs_client.status(compressed_path)['length']
        
        print(f"📊 Tamaño original: {original_size} bytes")
        print(f"📊 Tamaño comprimido: {compressed_size} bytes")
        print(f"📊 Ratio de compresión: {original_size/compressed_size:.2f}x")
    else:
        print(f"❌ Error: {result.stderr}")
        
except Exception as e:
    print(f"❌ Error: {e}")

print("\n✅ Compresión completada")

## 📝 EJERCICIO 3: Monitoreo y Recuperación

Implementaremos técnicas de monitoreo y recuperación

In [ ]:
# 3.1 Monitorear estado del cluster
print("📊 MONITOREANDO ESTADO DEL CLUSTER")
print("=============================")

try:
    # Obtener métricas del cluster
    result = subprocess.run(['hdfs', 'dfsadmin', '-report'], 
                          capture_output=True, text=True)
    
    if result.returncode == 0:
        # Extraer métricas importantes
        lines = result.stdout.split('\n')
        metrics = {}
        
        for line in lines:
            if 'Live datanodes' in line:
                metrics['datanodes'] = line.split(':')[1].strip()
            elif 'Dead datanodes' in line:
                metrics['dead_nodes'] = line.split(':')[1].strip()
            elif 'Configured Capacity' in line:
                metrics['capacity'] = line.split(':')[1].strip()
            elif 'DFS Used' in line:
                metrics['used'] = line.split(':')[1].strip()
        
        print("📊 Métricas del cluster:")
        for key, value in metrics.items():
            print(f"  - {key}: {value}")
    else:
        print(f"❌ Error: {result.stderr}")
        
except Exception as e:
    print(f"❌ Error: {e}")

print("\n✅ Monitoreo completado")

In [ ]:
# 3.2 Implementar recuperación
print("🔄 IMPLEMENTANDO RECUPERACIÓN")
print("=========================")

try:
    # Verificar integridad
    result = subprocess.run(['hdfs', 'fsck', '/', '-files', '-blocks', '-locations'], 
                          capture_output=True, text=True)
    
    if result.returncode == 0:
        print("✅ Verificación de integridad completada")
        
        # Crear punto de recuperación
        backup_dir = f'{analysis_dir}/backup_{datetime.now().strftime("%Y%m%d_%H%M%S")}'
        hdfs_client.makedirs(backup_dir)
        
        # Copiar archivos importantes
        hdfs_client.copy(hdfs_path, f'{backup_dir}/test_large.txt')
        hdfs_client.copy(compressed_path, f'{backup_dir}/test_compressed.gz')
        
        print(f"✅ Punto de recuperación creado en {backup_dir}")
    else:
        print(f"❌ Error: {result.stderr}")
        
except Exception as e:
    print(f"❌ Error: {e}")

print("\n✅ Recuperación implementada")

## 📝 LIMPIEZA FINAL

Limpiaremos los recursos utilizados

In [ ]:
# Limpiar recursos
print("🧹 LIMPIANDO RECURSOS")
print("===================")

try:
    # Eliminar archivos locales
    if os.path.exists(test_file):
        os.remove(test_file)
        print(f"✅ Archivo local {test_file} eliminado")
    
    # Eliminar directorio de análisis
    hdfs_client.delete(analysis_dir, recursive=True)
    print(f"✅ Directorio {analysis_dir} eliminado de HDFS")
    
except Exception as e:
    print(f"❌ Error: {e}")

print("\n✅ Limpieza completada")